<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-18 14:43:31
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  4.98 L
Current:  1.43 C
-------------------
Today PnL: -11.43 K (-0.08%)
Current PnL: -31.41 L (-19.63%)
CY Booked + Current PnL: -15.73 L (-9.83%)
-------------------
Total profit:  1.33 L
Total loss:  -32.74 L
-------------------
Total Booked + Current PnL: 11.25 L (8.46%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (10.96%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.40 C
Est FTT PnL: 96.75 L (67.64%)
Deployed:  1.33 C
Current:  1.43 C
CAGR/XIRR %: 3.77%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,0.13,3.57,17.38,21.57,17636.0,3495.0,101475.0,397.04,-10.57,60.0,H-SC,4.90,167.0,0.20,0.74,48.08,XR,ATH,FINANCE
18,COALINDIA,0.19,2.51,14.79,17.67,22811.0,3774.0,154234.0,478.51,23.39,49.0,L-LC,11.35,185.0,0.17,1.12,23.81,XY25,ATH,MINING
77,TTKPRESTIG,-0.24,-27.49,38.03,0.09,27792.0,-27699.0,73078.0,770.00,69.69,39.0,M-SC,9.36,250.0,-1.00,0.53,0.58,OX40N,NTT,DURABLES
36,ICICIGI,0.87,3.03,15.83,19.34,32637.0,6061.0,206170.0,2252.93,-13.21,63.0,X-MC,2.86,66.0,0.19,1.49,19.45,X40,ATH,INSURANCE
51,MEDANTA,0.04,-2.07,26.69,24.08,33183.0,-2623.0,124327.0,1486.00,-2.82,58.0,X-SC,11.02,87.0,-0.08,0.90,13.87,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,LAOPALA,3.27,-40.47,133.64,39.09,104306.0,-53055.0,78050.0,464.00,70.57,50.0,H-SC,15.26,154.0,-0.51,0.57,4.37,AR,NTT,CERAMICS
20,COLPAL,2.31,-16.44,69.35,41.51,152619.0,-43295.0,220070.0,3726.84,1.10,61.0,X-MC,6.21,62.0,-0.28,1.59,7.53,XY25,ATH,FMCG
25,EASEMYTRIP,2.00,-46.52,172.45,45.70,163060.0,-82260.0,94555.0,26.40,-1.77,82.0,M-SC,22.93,204.0,-0.50,0.68,55.79,XY24,NTT,TRAVEL
44,ITC,1.97,-18.67,36.21,10.78,70776.0,-44858.0,195460.0,452.00,-51.88,55.0,X-LC,18.66,1.0,-0.63,1.42,7.66,X40,NTT,FMCG
80,VALIANTORG,1.87,-48.83,240.32,74.15,201201.0,-79883.0,83722.0,838.00,-250.34,43.0,H-SC,24.09,145.0,-0.40,0.61,7.32,XR,NTT,CHEMICALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BSOFT,-2.85,-32.04,121.88,50.79,116501.0,-45066.0,95587.0,831.70,-12.64,36.0,H-SC,7.00,172.0,-0.39,0.69,11.29,XR,ATH,IT
39,INDIAMART,-2.55,-6.88,121.99,106.72,140107.0,-8485.0,114851.0,4810.62,-58.01,43.0,H-SC,7.81,140.0,-0.06,0.83,14.89,AR,ATH,MISC
84,WIPRO,-2.07,-13.36,98.84,72.27,145102.0,-22643.0,146805.0,420.00,-21.74,26.0,M-LC,6.45,98.0,-0.16,1.06,0.00,XR,NTT,IT
30,HAPPSTMNDS,-1.69,-51.76,283.21,84.85,175092.0,-66341.0,61824.0,1480.71,-42.72,39.0,H-SC,25.64,152.0,-0.38,0.45,3.90,AR,ATH,IT
56,RAJOOENG,-1.65,-36.15,118.64,39.61,77650.0,-37050.0,65450.0,143.10,-56.80,49.0,H-SC,28.63,136.0,-0.48,0.47,9.65,AR,ATH,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,0.32,4.02,99.87,107.91,153383.0,5939.0,153583.0,1641.55,-6.65,65.0,M-SC,8.79,220.0,0.04,1.11,15.55,OX40N,ATH,CABLES
77,TTKPRESTIG,-0.24,-27.49,38.03,0.09,27792.0,-27699.0,73078.0,770.00,69.69,39.0,M-SC,9.36,250.0,-1.00,0.53,0.58,OX40N,NTT,DURABLES
66,SIS,0.60,-28.44,69.45,21.26,55184.0,-31574.0,79458.0,528.00,1868.91,37.0,H-SC,4.23,168.0,-0.57,0.58,9.54,OX40N,NTT,MISC
72,TATAELXSI,-1.16,-29.93,88.42,32.03,81681.0,-39457.0,92378.0,9161.00,-12.30,35.0,H-SC,7.25,158.0,-0.48,0.67,3.47,OX40N,NTT,IT
40,INDIGOPNTS,-0.31,-30.74,44.41,0.02,53692.0,-53659.0,120900.0,1408.00,84.15,30.0,M-SC,8.68,222.0,-1.00,0.88,5.16,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,0.19,2.51,14.79,17.67,22811.0,3774.0,154234.0,478.51,23.39,49.0,L-LC,11.35,185.0,0.17,1.12,23.81,XY25,ATH,MINING
26,FINCABLES,0.32,4.02,99.87,107.91,153383.0,5939.0,153583.0,1641.55,-6.65,65.0,M-SC,8.79,220.0,0.04,1.11,15.55,OX40N,ATH,CABLES
50,MASFIN,0.13,3.57,17.38,21.57,17636.0,3495.0,101475.0,397.04,-10.57,60.0,H-SC,4.90,167.0,0.20,0.74,48.08,XR,ATH,FINANCE
1,ABB,0.27,7.37,35.58,45.57,99943.0,19277.0,280896.0,7934.00,-31.00,68.0,H-MC,4.31,125.0,0.19,2.03,24.74,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,0.13,3.57,17.38,21.57,17636.0,3495.0,101475.0,397.04,-10.57,60.0,H-SC,4.90,167.0,0.20,0.74,48.08,XR,ATH,FINANCE
1,ABB,0.27,7.37,35.58,45.57,99943.0,19277.0,280896.0,7934.00,-31.00,68.0,H-MC,4.31,125.0,0.19,2.03,24.74,AR,NTT,ELECTRICAL
26,FINCABLES,0.32,4.02,99.87,107.91,153383.0,5939.0,153583.0,1641.55,-6.65,65.0,M-SC,8.79,220.0,0.04,1.11,15.55,OX40N,ATH,CABLES
39,INDIAMART,-2.55,-6.88,121.99,106.72,140107.0,-8485.0,114851.0,4810.62,-58.01,43.0,H-SC,7.81,140.0,-0.06,0.83,14.89,AR,ATH,MISC
85,ZYDUSLIFE,-0.06,-4.45,41.95,35.63,84004.0,-9326.0,200248.0,1286.17,-16.67,51.0,H-MC,5.43,120.0,-0.11,1.45,11.48,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,-1.30,-6.53,43.62,34.24,127575.0,-20418.0,292470.0,1972.00,-26.34,24.0,X-LC,3.95,3.0,-0.16,2.12,1.22,X40,NTT,IT
73,TCS,-1.07,-26.08,60.38,18.55,168812.0,-98654.0,279583.0,4311.59,-33.14,26.0,X-LC,4.22,2.0,-0.58,2.03,0.00,X40,ATH,IT
33,HCLTECH,-1.21,-4.95,30.28,23.83,69631.0,-11966.0,229958.0,1908.19,-0.33,27.0,X-LC,4.54,8.0,-0.17,1.67,10.33,X40,ATH,IT
8,AWL,-0.72,-34.34,139.41,57.19,298514.0,-112000.0,214127.0,485.00,-62.47,29.0,X-SC,17.56,81.0,-0.38,1.55,0.50,XY24,NTT,FMCG
13,BERGEPAINT,-0.64,-17.97,47.68,21.14,88915.0,-40857.0,186482.0,680.00,-19.82,30.0,X-MC,12.43,68.0,-0.46,1.35,0.43,XY24,NTT,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,-0.24,4.36,15.73,20.77,38435.0,10199.0,244343.0,1644.00,-7.40,63.0,X-MC,1.47,64.0,0.27,1.77,12.08,X40N,NTT,BREWERIES
21,DABUR,-1.41,-0.39,44.26,43.70,109370.0,-960.0,247108.0,735.00,-9.68,48.0,X-MC,1.88,70.0,-0.01,1.79,14.61,XY24,BTT,FMCG
65,SIEMENS,0.00,-14.39,46.61,25.51,74259.0,-26775.0,159320.0,4671.50,33.78,56.0,H-LC,1.95,51.0,-0.36,1.15,17.41,AR,ATH,ELECTRICAL
34,HINDUNILVR,0.59,-9.52,25.62,13.67,59592.0,-24463.0,232600.0,2922.00,-32.77,43.0,X-LC,2.86,9.0,-0.41,1.68,9.81,XY25,NTT,FMCG
36,ICICIGI,0.87,3.03,15.83,19.34,32637.0,6061.0,206170.0,2252.93,-13.21,63.0,X-MC,2.86,66.0,0.19,1.49,19.45,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,-1.07,-26.08,60.38,18.55,168812.0,-98654.0,279583.0,4311.59,-33.14,26.0,X-LC,4.22,2.0,-0.58,2.03,0.00,X40,ATH,IT
3,ACC,-0.11,-31.18,138.52,64.15,226840.0,-74191.0,163760.0,3906.00,-48.05,39.0,X-SC,8.48,85.0,-0.33,1.19,0.33,XY24,BTT,CEMENT
53,PGHH,0.09,-13.05,51.83,32.01,96221.0,-27872.0,185648.0,17616.87,-41.22,36.0,X-MC,9.31,57.0,-0.29,1.34,0.34,X40,ATH,FMCG
13,BERGEPAINT,-0.64,-17.97,47.68,21.14,88915.0,-40857.0,186482.0,680.00,-19.82,30.0,X-MC,12.43,68.0,-0.46,1.35,0.43,XY24,NTT,PAINTS
8,AWL,-0.72,-34.34,139.41,57.19,298514.0,-112000.0,214127.0,485.00,-62.47,29.0,X-SC,17.56,81.0,-0.38,1.55,0.50,XY24,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,-0.00,-30.06,104.23,42.84,47390.0,-19539.0,45467.0,424.00,-54.27,52.0,X-SC,18.85,80.0,-0.41,0.33,9.37,XY24,NTT,MISC
12,BATAINDIA,-0.39,-46.11,150.87,35.20,104622.0,-59324.0,69346.0,2096.00,-9.03,37.0,X-SC,21.52,91.0,-0.57,0.50,0.80,X40,NTT,FOOTWEAR
58,RELAXO,0.77,-51.95,216.98,52.31,151339.0,-75412.0,69748.0,1176.00,-48.89,47.0,X-SC,14.05,92.0,-0.50,0.51,4.57,X40N,NTT,FOOTWEAR
51,MEDANTA,0.04,-2.07,26.69,24.08,33183.0,-2623.0,124327.0,1486.00,-2.82,58.0,X-SC,11.02,87.0,-0.08,0.90,13.87,XY24,NTT,HEALTHCARE
35,HONAUT,0.08,-22.88,85.85,43.32,107828.0,-37272.0,125600.0,58357.33,-32.25,39.0,X-SC,11.15,90.0,-0.35,0.91,1.98,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,1.97,-18.67,36.21,10.78,70776.0,-44858.0,195460.0,452.00,-51.88,55.0,X-LC,18.66,1.0,-0.63,1.42,7.66,X40,NTT,FMCG
73,TCS,-1.07,-26.08,60.38,18.55,168812.0,-98654.0,279583.0,4311.59,-33.14,26.0,X-LC,4.22,2.0,-0.58,2.03,0.00,X40,ATH,IT
42,INFY,-1.30,-6.53,43.62,34.24,127575.0,-20418.0,292470.0,1972.00,-26.34,24.0,X-LC,3.95,3.0,-0.16,2.12,1.22,X40,NTT,IT
75,TMPV,-0.17,-21.66,56.99,22.98,100849.0,-48939.0,176959.0,600.00,-81.44,63.0,X-LC,2.95,4.0,-0.49,1.28,13.13,XY24,NTT,AUTO
33,HCLTECH,-1.21,-4.95,30.28,23.83,69631.0,-11966.0,229958.0,1908.19,-0.33,27.0,X-LC,4.54,8.0,-0.17,1.67,10.33,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,0.26,-30.14,90.21,32.89,48658.0,-23268.0,53939.0,1800.00,-354.20,65.0,L-MC,11.14,258.0,-0.48,0.39,48.54,XR,NTT,BANKS
67,SONACOMS,0.02,-8.34,51.74,39.09,47975.0,-8435.0,92724.0,804.02,-27.36,63.0,M-MC,6.68,193.0,-0.18,0.67,31.17,AR,ATH,AUTO
25,EASEMYTRIP,2.00,-46.52,172.45,45.70,163060.0,-82260.0,94555.0,26.40,-1.77,82.0,M-SC,22.93,204.0,-0.50,0.68,55.79,XY24,NTT,TRAVEL
50,MASFIN,0.13,3.57,17.38,21.57,17636.0,3495.0,101475.0,397.04,-10.57,60.0,H-SC,4.90,167.0,0.20,0.74,48.08,XR,ATH,FINANCE
79,VAIBHAVGBL,-1.49,-23.63,110.51,60.77,154254.0,-43191.0,139584.0,521.00,60.88,56.0,H-SC,1.47,160.0,-0.28,1.01,26.55,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,2.00,-46.52,172.45,45.70,163060.0,-82260.0,94555.0,26.40,-1.77,82.0,M-SC,22.93,204.0,-0.50,0.68,55.79,XY24,NTT,TRAVEL
7,ATULAUTO,1.50,-9.93,61.61,45.57,109398.0,-19567.0,177565.0,844.00,3851.35,70.0,M-SC,4.38,248.0,-0.18,1.29,34.38,XY24,NTT,AUTO
11,BANDHANBNK,1.87,-17.46,133.13,92.42,306089.0,-48642.0,229917.0,400.00,97.56,71.0,H-SC,7.03,174.0,-0.16,1.67,32.87,XY24,NTT,BANKS
1,ABB,0.27,7.37,35.58,45.57,99943.0,19277.0,280896.0,7934.00,-31.00,68.0,H-MC,4.31,125.0,0.19,2.03,24.74,AR,NTT,ELECTRICAL
67,SONACOMS,0.02,-8.34,51.74,39.09,47975.0,-8435.0,92724.0,804.02,-27.36,63.0,M-MC,6.68,193.0,-0.18,0.67,31.17,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.19
1,25,44.68
2,50,76.79


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.37
MC    30.34
LC    23.30
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.90
X40      23.20
X40N     15.21
AR        8.99
XY25      8.84
XR        8.70
OX40N     7.30
SR        0.87
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    22.88
H-SC    22.70
X-LC    19.97
X-SC    11.86
M-SC    11.03
H-MC     4.95
M-MC     2.12
H-LC     1.15
L-LC     1.12
M-LC     1.06
L-SC     0.78
L-MC     0.39
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.09,-13.64,51.82
FINANCE,13.45,-12.44,56.69
IT,11.45,-35.37,102.92
MISC,7.04,-33.51,88.12
ELECTRICAL,6.27,-8.30,47.81
PAINTS,5.10,-31.39,50.91
INSURANCE,4.87,-0.85,35.36
PHARMA,4.10,-3.92,36.63
AUTO,3.24,-17.20,57.74


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3257578.0,21
AR,1329024.0,10
X40,1295672.0,15
XR,1290025.0,12
X40N,1022314.0,10
OX40N,784401.0,10
XY25,396756.0,6
SR,299255.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3662020.0,24
M-SC,1393162.0,14
X-MC,1354495.0,15
X-SC,1167065.0,10
X-LC,1136621.0,12
H-MC,395153.0,3
L-SC,182633.0,2
M-LC,145102.0,1
M-MC,93046.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1237088.0      6
           AR         948937.0      5
           XR         829555.0      7
M-SC       XY24       797963.0      6
X-SC       XY24       605927.0      4
X-LC       X40        579245.0      6
X-MC       X40        541517.0      7
           X40N       413643.0      4
X-SC       X40N       386228.0      4
H-SC       OX40N      347185.0      4
M-SC       OX40N      344360.0      5
H-SC       SR         299255.0      2
X-LC       X40N       222443.0      2
H-MC       XY24       211206.0      1
X-LC       XY24       207109.0      2
X-MC       XY25       201050.0      2
           XY24       198285.0      2
H-MC       AR         183947.0      2
M-SC       XR         176933.0      2
X-SC       X40        174910.0      2
M-LC       XR         145102.0      1
X-LC       XY25       127824.0      2
L-SC       OX40N       92856.0      1
           XR          89777.0      1
H-LC       AR          74259.0      1
M-SC       AR          73906.0      1
L-MC       XR          48658.0      1
M-MC       AR          47975.0      1
           XY25        45071.0      1
L-LC       XY25        22811.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 34.0 seconds
